<a href="https://colab.research.google.com/github/GavinHacker/recsys_model/blob/master/7_recbaserecall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用基于电影相似度进行推荐的方法进行召回

### install library

In [0]:
!pip install pymysql

    100% |████████████████████████████████| 51kB 1.8MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Functional code

In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce
import numpy as np
import pandas as pd
import uuid
import datetime
#from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import pairwise_distances
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
from sklearn.datasets import dump_svmlight_file
from sklearn.preprocessing import OneHotEncoder
import pickle as pkl
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression

In [0]:
def get_connection():
    return pymysql.connect(host='rm-2zeqqm6994abi7b6dqo.mysql.rds.aliyuncs.com',
                               user='noone',
                               password='Huawei12#$',
                               db='recsys',
                               port=3306,
                               charset ='utf8',
                               use_unicode=True)

## 获取最新的comment，实际recsys_core实现的时候使用mqlog消息

In [0]:
def get_comment_data():
  df_comment_new_data = pd.read_sql_query("select * from comment_new where newdata = 1 ", get_connection())
  df_comment_new_data_ldim = df_comment_new_data.loc[:,['ID','MOVIEID','USERID']]
  return df_comment_new_data_ldim

## 获取基于相似度推荐的电影集合

In [0]:
def get_ibmovie_by_movieid(movieid, connection):
  sql = 'select DISTINCT recmovieid from ibmovie where movieid = \'%s\'' % movieid
  try:
      with connection.cursor() as cursor:
          cout=cursor.execute(sql)
      return cursor.fetchall()
  except Exception as e:
      print(e)
      connection.close()
  return None

## 对recmovie表插入数据，保留原始movieid，即根据哪个电影推荐而来

In [0]:
def insert_or_update_recmovie(movieid, userid, srcmovieid, connection):
  _id = uuid.uuid4()
  time_now = datetime.datetime.now()
  q_sql = 'select id from recmovie where userid=\'%s\' and movieid=\'%s\'' % (userid, movieid)
  i_sql = 'insert into recmovie (id, userid, movieid, rectime, srcmovieid) values (\'%s\', \'%s\', \'%s\', \'%s\', \'%s\')' % (_id, userid, movieid, time_now, srcmovieid)
  exist_list = None
  try:
      with connection.cursor() as cursor:
        #print(q_sql)
        cout=cursor.execute(q_sql)
        exist_list = cursor.fetchall()
      if len(exist_list) > 0:
        with connection.cursor() as cursor:
          for item in exist_list:
              u_sql = 'update recmovie set rectime=\'%s\' where id=\'%s\'' % (time_now, item[0])
              cursor.execute(u_sql)
      else:
        with connection.cursor() as cursor:
              cursor.execute(i_sql)
  except Exception as e:
      print(e)
      connection.close()
  return None

* test code
    
    insert_or_update_recmovie('10430817','cf2349f9c01f9a5cd4050aebd30ab74f',conn)

In [0]:
def update_comment_new_data_flag(rid, connection):
  sql = 'update comment_new set newdata = 0 where id = \'%s\'' % rid
  try:
      with connection.cursor() as cursor:
          cout=cursor.execute(sql)
  except Exception as e:
      print(e)
      connection.close()

## 根据用户的打分进行相似电影召回，暂不限数量，具体召回数量取决于用户看过的电影的相似电影集合交集

In [0]:
def func_main():
  df_comment_new_data_ldim = get_comment_data()
  conn = get_connection()
  for i in df_comment_new_data_ldim.index:
    print(df_comment_new_data_ldim.iloc[i]['MOVIEID'], df_comment_new_data_ldim.iloc[i]['USERID'])
    ibmovie_list = get_ibmovie_by_movieid(df_comment_new_data_ldim.iloc[i]['MOVIEID'], get_connection())
    for j in ibmovie_list:
      insert_or_update_recmovie(j[0],'cf2349f9c01f9a5cd4050aebd30ab74f', df_comment_new_data_ldim.iloc[i]['MOVIEID'], conn)
    update_comment_new_data_flag(df_comment_new_data_ldim.iloc[i]['ID'], conn)
    conn.commit()

In [0]:
#func_main()